# Pricing d'options asiatiques arithmétiques en modèle binomial

Objectifs du TP :

1. Télécharger et analyser des données BTC / ETH (prix, log-rendements, volatilité).
2. Implémenter un **binomial tree naïf** pour une option asiatique arithmétique.
3. Comprendre pourquoi ce schéma explose en complexité (2^N).
4. Implémenter un **binomial amélioré type Hull–White** (réduction de dimension).
5. Étudier les sensibilités du prix asiatique à :
   - la maturité,
   - le nombre de pas,
   - le spot,
   - le strike.
6. Comparer **option asiatique** vs **option européenne** (Black–Scholes).

On travaille avec :

- Options **asiatiques arithmétiques à strike fixe**.
- Sous-jacent : BTC (avec données Yahoo Finance).
- Modèle : sous-jacent log-normal, volatilité constante (Black–Scholes).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as wb
import time
from scipy.stats import norm

plt.style.use("ggplot")


In [ ]:
# 1. Récupération des données de prix BTC et ETH via Yahoo Finance

start_date = "2018-01-01"

BTC = wb.DataReader("BTC-USD", data_source="yahoo", start=start_date)
ETH = wb.DataReader("ETH-USD", data_source="yahoo", start=start_date)

price_BTC = BTC["Adj Close"]
price_ETH = ETH["Adj Close"]

BTC.info()
ETH.info()


In [ ]:
# 1.1. Prix bruts BTC vs ETH

plt.figure(figsize=(10, 5))
plt.plot(price_BTC, color="red", label="BTC-USD")
plt.plot(price_ETH, color="blue", label="ETH-USD")
plt.title("Prix spot BTC-USD et ETH-USD")
plt.xlabel("Date")
plt.ylabel("Prix")
plt.legend()
plt.show()

# 1.2. Deux axes Y pour comparer les ordres de grandeur

fig, ax1 = plt.subplots(figsize=(10, 5))

ax1.set_xlabel("Date")
ax1.set_ylabel("BTC", color="red")
ax1.plot(price_BTC, color="red")
ax1.tick_params(axis="y", labelcolor="red")

ax2 = ax1.twinx()
ax2.set_ylabel("ETH", color="blue")
ax2.plot(price_ETH, color="blue")
ax2.tick_params(axis="y", labelcolor="blue")

fig.tight_layout()
plt.title("BTC vs ETH sur deux axes")
plt.show()

# 1.3. Prix normalisés (centrés-réduits) pour comparer les dynamiques

btc_norm = (price_BTC - price_BTC.mean()) / price_BTC.std()
eth_norm = (price_ETH - price_ETH.mean()) / price_ETH.std()

plt.figure(figsize=(10, 5))
plt.plot(btc_norm, color="red", label="BTC normalisé")
plt.plot(eth_norm, color="blue", label="ETH normalisé")
plt.title("Prix normalisés BTC / ETH")
plt.xlabel("Date")
plt.ylabel("Valeur normalisée")
plt.legend()
plt.show()


In [ ]:
# 2. Analyse des log-rendements

log_ret_BTC = np.log(price_BTC).diff().dropna()
log_ret_ETH = np.log(price_ETH).diff().dropna()

print("Moyenne log-ret BTC  :", log_ret_BTC.mean())
print("Moyenne log-ret ETH  :", log_ret_ETH.mean())
print("Volatilité BTC (quotidienne) :", log_ret_BTC.std())
print("Volatilité ETH (quotidienne) :", log_ret_ETH.std())


## 3. Binomial tree naïf pour option asiatique (BTM)

Idée :

- On considère un arbre binomial classique :
  - à chaque pas, le prix monte de `u` ou descend de `d`.
- Pour une option asiatique **arithmétique**, le payoff dépend de la moyenne :
  \
  A = \frac{1}{N+1}\sum_{n=0}^N S_n
  \
- Le code naïf ci-dessous construit **toutes les trajectoires possibles** jusqu'à l'échéance :
  - `St` contient les `2^N` valeurs terminales de `S_N` (tous les chemins),
  - `At` accumule la somme le long de chaque trajectoire,
  - puis on divise par `N+1` pour obtenir la moyenne arithmétique.

Complexité : **O(2^N)** → impraticable pour de gros N.


In [ ]:
def BTM(strike_type, option_type, S0, K, r, sigma, T, N):
    """
    Binomial Tree Model naïf pour option asiatique arithmétique.

    strike_type : "fixed" (strike K) ou "floating" (strike = moyenne / S_N)
    option_type : "C" (call) ou "P" (put)
    S0          : spot initial
    K           : strike
    r           : taux sans risque (continu)
    sigma       : volatilité
    T           : maturité
    N           : nombre de pas de temps
    """
    deltaT = T / N
    u = np.exp(sigma * np.sqrt(deltaT))
    d = 1.0 / u
    p = (np.exp(r * deltaT) - d) / (u - d)

    # On part de S0
    St = [S0]
    At = [S0]     # somme des prix le long du chemin
    strike = [K]  # strike répété (cas fixed strike)

    # Construction exhaustive des 2^N trajectoires
    for _ in range(N):
        # Nouveaux prix terminal pour tous les chemins :
        # - branche up : S * u
        # - branche down : S * d
        St = [s * u for s in St] + [s * d for s in St]

        # Duplique les chemins pour les deux branches (up et down)
        At = At + At
        strike = strike + strike

        # Ajoute le prix courant au cumul de chaque trajectoire
        for i in range(len(At)):
            At[i] = At[i] + St[i]

    # Moyenne arithmétique sur chaque trajectoire
    At = np.array(At) / (N + 1)
    St = np.array(St)
    strike = np.array(strike)

    # Payoff asiatique (strike fixe ou flottant)
    if strike_type == "fixed":
        if option_type == "C":
            payoff = np.maximum(At - strike, 0.0)
        else:
            payoff = np.maximum(strike - At, 0.0)
    else:
        # floating strike
        if option_type == "C":
            payoff = np.maximum(St - At, 0.0)
        else:
            payoff = np.maximum(At - St, 0.0)

    # Remontée backward sur l'arbre (probabilité neutre au risque)
    option_price = payoff.copy()
    for _ in range(N):
        length = len(option_price) // 2
        option_price = p * option_price[:length] + (1 - p) * option_price[length:]

    return option_price[0]


In [ ]:
# 4. Étude de sensibilité à la maturité (avec N fixed petit, sinon trop lent)

range_T = [1/365, 1/12, 1/4, 1/2, 1, 2, 3]
prices_BT = []

for T_val in range_T:
    price = BTM(
        strike_type="fixed",
        option_type="C",
        S0=57830,
        K=58000,
        r=0.01,
        sigma=0.05,
        T=T_val,
        N=10  # volontairement petit
    )
    prices_BT.append(price)

plt.figure(figsize=(10, 5))
plt.plot(range_T, prices_BT, marker="o")
plt.xlabel("Maturité T (années)")
plt.ylabel("Prix option asiatique (BTM naïf)")
plt.title("Sensibilité à la maturité (BTM naïf)")
plt.grid(True)
plt.show()


In [ ]:
# 5. Sensibilité au nombre de pas + temps de calcul (BTM naïf)
range_N = np.array([2, 3, 4, 5] + [6 + j for j in range(15)])  # N monte doucement
prices_N = []
times_N = []

for N_val in range_N:
    start_time = time.time()
    price = BTM(
        strike_type="fixed",
        option_type="C",
        S0=57830,
        K=58000,
        r=0.01,
        sigma=0.05,
        T=1.0,
        N=N_val
    )
    elapsed = time.time() - start_time
    prices_N.append(price)
    times_N.append(elapsed)
    print(f"N={N_val:3d} → prix={price:.4f}, temps={elapsed:.3f} s")

plt.figure(figsize=(10, 5))
plt.plot(range_N, prices_N, marker="o")
plt.xlabel("Nombre de pas N")
plt.ylabel("Prix option asiatique (BTM naïf)")
plt.title("Sensibilité au nombre de pas (BTM naïf)")
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(range_N, times_N, marker="o", color="red")
plt.xlabel("Nombre de pas N")
plt.ylabel("Temps de calcul (s)")
plt.title("Explosion de la complexité (BTM naïf)")
plt.grid(True)
plt.show()


## 6. Binomial “amélioré” type Hull–White (HW_BTM)

Problème du BTM naïf : la moyenne arithmétique dépend de tout le chemin,
et le nombre d'états possibles explose (2^N).

Idée Hull–White :

- On n'utilise plus tous les chemins, mais une grille **finie** de valeurs possibles de moyenne `A` à chaque nœud (N, J).
- À chaque couple (N, J), on approxime les possibles valeurs d'average `A` par M points entre un `A_min` et un `A_max`.
- On interpole les payoffs entre ces valeurs d'average lors de la remontée backward.

Résultat : la complexité devient O(N² M), beaucoup plus raisonnable.
Mais le code devient plus lourd : gestion de grilles `A` et interpolation linéaire.

On reprend ton implémentation, en la commentant pour la rendre lisible.


In [ ]:
def HW_BTM(strike_type, option_type, S0, K, r, sigma, T, step, M):
    """
    Schéma binomial de type Hull–White pour option asiatique arithmétique.

    strike_type : "fixed" ou "floating"
    option_type : "C" (call) ou "P" (put)
    S0          : spot
    K           : strike
    r           : taux
    sigma       : volatilité
    T           : maturité
    step        : nombre de pas de temps N
    M           : nombre de points pour discrétiser la moyenne A à chaque nœud

    Idée :
    - Pour chaque niveau (N, J), on approxime la moyenne par M valeurs
      allant de A_min à A_max.
    - À la remontée backward, on interpole les payoffs en fonction de A.
    """

    N = step
    deltaT = T / N
    u = np.exp(sigma * np.sqrt(deltaT))
    d = 1.0 / u
    p = (np.exp(r * deltaT) - d) / (u - d)

    # On va construire At[N][J][k] = valeur de moyenne approx à l'état (N,J) et point k
    At = []
    strike = np.array([K] * M)

    # 1. Construction de la grille de moyennes At au temps final N
    for J in range(N + 1):
        # On calcule A_max et A_min possibles à (N,J) en utilisant deux chemins extrêmes
        # (toutes les hausses d'abord / toutes les baisses d'abord)
        path_up_then_down = np.array(
            [S0 * u**j * d**0 for j in range(N - J)] +
            [S0 * u**(N - J) * d**j for j in range(J + 1)]
        )
        A_max = path_up_then_down.mean()

        path_down_then_up = np.array(
            [S0 * d**j * u**0 for j in range(J + 1)] +
            [S0 * d**J * u**(j + 1) for j in range(N - J)]
        )
        A_min = path_down_then_up.mean()

        diff = A_max - A_min
        # M points régulièrement espacés entre A_max et A_min
        A_vals = [A_max - diff * k / (M - 1) for k in range(M)]
        At.append(A_vals)

    At = np.round(At, 4)
    St = np.array([S0 * u**(N - J) * d**J for J in range(N + 1)])  # S à l'échéance selon J

    # 2. Payoff à l'échéance pour chaque (N,J,k)
    # On utilise la même moyenne At pour toutes les J au début
    payoff = []

    for J in range(N + 1):
        A_vals = np.array(At[J])
        S_vals = np.array([S0 * u**(N - J) * d**J] * M)

        if strike_type == "fixed":
            if option_type == "C":
                pay = np.maximum(A_vals - strike, 0.0)
            else:
                pay = np.maximum(strike - A_vals, 0.0)
        else:
            if option_type == "C":
                pay = np.maximum(S_vals - A_vals, 0.0)
            else:
                pay = np.maximum(A_vals - S_vals, 0.0)

        payoff.append(pay)

    payoff = np.round(np.array(payoff), 4)

    # 3. Remontée backward avec interpolation sur A
    for n in range(N - 1, -1, -1):
        At_backward = []
        payoff_backward = []

        for J in range(n + 1):
            # Recalcule les A_min / A_max pour le niveau (n,J)
            path_up_then_down = np.array(
                [S0 * u**j * d**0 for j in range(n - J)] +
                [S0 * u**(n - J) * d**j for j in range(J + 1)]
            )
            A_max = path_up_then_down.mean()

            path_down_then_up = np.array(
                [S0 * d**j * u**0 for j in range(J + 1)] +
                [S0 * d**J * u**(j + 1) for j in range(n - J)]
            )
            A_min = path_down_then_up.mean()

            diff = A_max - A_min
            A_vals = np.array([A_max - diff * k / (M - 1) for k in range(M)])
            At_backward.append(A_vals)

        At_backward = np.round(np.array(At_backward), 4)

        # On va construire pour chaque (n,J,k) la valeur en fonction des états enfants (up, down)
        payoff_new = []

        for J in range(n + 1):
            A_vals = At_backward[J]
            pay_vals = np.zeros_like(A_vals)

            # Enfant "up" est (n+1, J), enfant "down" est (n+1, J+1)
            A_up = np.array(At[J])
            A_down = np.array(At[J + 1])
            pay_up = payoff[J]
            pay_down = payoff[J + 1]

            # Interpolation linéaire de pay_up(A) et pay_down(A) sur les nouvelles A_vals
            for k, A_k in enumerate(A_vals):
                # interpolation sur l'enfant up
                if A_k <= A_up[0]:
                    fu = pay_up[0]
                elif A_k >= A_up[-1]:
                    fu = pay_up[-1]
                else:
                    idx = np.searchsorted(A_up, A_k) - 1
                    x0, x1 = A_up[idx], A_up[idx+1]
                    y0, y1 = pay_up[idx], pay_up[idx+1]
                    fu = y0 + (y1 - y0) * (A_k - x0) / (x1 - x0)

                # interpolation sur l'enfant down
                if A_k <= A_down[0]:
                    fd = pay_down[0]
                elif A_k >= A_down[-1]:
                    fd = pay_down[-1]
                else:
                    idx = np.searchsorted(A_down, A_k) - 1
                    x0, x1 = A_down[idx], A_down[idx+1]
                    y0, y1 = pay_down[idx], pay_down[idx+1]
                    fd = y0 + (y1 - y0) * (A_k - x0) / (x1 - x0)

                # Valeur actualisée au nœud (n,J,A_k)
                pay_vals[k] = (p * fu + (1 - p) * fd) * np.exp(-r * deltaT)

            payoff_backward.append(pay_vals)

        # On remplace At et payoff par les valeurs du niveau précédent
        At = At_backward
        payoff = np.round(np.array(payoff_backward), 4)

    # Au temps 0, il reste une seule ligne J=0, on en prend la moyenne sur A
    option_price = payoff[0].mean()
    return option_price


In [ ]:
range_T = [1/365, 1/12, 1/4, 1/2, 1, 2, 3]
prices_hw_T = []

for T_val in range_T:
    price = HW_BTM(
        strike_type="fixed",
        option_type="C",
        S0=57830,
        K=58000,
        r=0.01,
        sigma=0.05,
        T=T_val,
        step=10,
        M=10
    )
    prices_hw_T.append(price)

plt.figure(figsize=(10, 5))
plt.plot(range_T, prices_hw_T, marker="o")
plt.xlabel("Maturité T (années)")
plt.ylabel("Prix asiatique (HW_BTM)")
plt.title("Sensibilité à la maturité (HW_BTM)")
plt.grid(True)
plt.show()


In [ ]:
range_N = np.array([2, 3, 4, 5] + [6 + j for j in range(50)])
prices_hw_N = []
times_hw_N = []

for N_val in range_N:
    start_time = time.time()
    price = HW_BTM(
        strike_type="fixed",
        option_type="P",
        S0=57830,
        K=58000,
        r=0.01,
        sigma=0.05,
        T=1.0,
        step=N_val,
        M=int(round(N_val / 3.5)) + 1
    )
    elapsed = time.time() - start_time
    prices_hw_N.append(price)
    times_hw_N.append(elapsed)
    print(f"N={N_val:3d} → prix={price:.4f}, temps={elapsed:.3f} s")

plt.figure(figsize=(10, 5))
plt.plot(range_N, prices_hw_N, marker="o")
plt.xlabel("Nombre de pas N")
plt.ylabel("Prix asiatique (HW_BTM)")
plt.title("Sensibilité au nombre de pas (HW_BTM)")
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(range_N, times_hw_N, marker="o", color="red")
plt.xlabel("Nombre de pas N")
plt.ylabel("Temps de calcul (s)")
plt.title("Coût de calcul HW_BTM vs N")
plt.grid(True)
plt.show()


In [ ]:
range_price = np.array([57830 * (0.95 + 0.01 * j) for j in range(10)])

# Sensibilité au spot
prices_vs_spot = []
for S_val in range_price:
    price = HW_BTM(
        strike_type="fixed",
        option_type="P",
        S0=S_val,
        K=58000,
        r=0.01,
        sigma=0.05,
        T=1.0,
        step=10,
        M=4
    )
    prices_vs_spot.append(price)

plt.figure(figsize=(10, 5))
plt.plot(range_price, prices_vs_spot, marker="o")
plt.xlabel("Spot S0")
plt.ylabel("Prix asiatique (HW_BTM)")
plt.title("Sensibilité au spot (put asiatique)")
plt.grid(True)
plt.show()

# Sensibilité au strike
prices_vs_strike = []
for K_val in range_price:
    price = HW_BTM(
        strike_type="fixed",
        option_type="C",
        S0=57830,
        K=K_val,
        r=0.01,
        sigma=0.05,
        T=1.0,
        step=10,
        M=4
    )
    prices_vs_strike.append(price)

plt.figure(figsize=(10, 5))
plt.plot(range_price, prices_vs_strike, marker="o")
plt.xlabel("Strike K")
plt.ylabel("Prix asiatique (HW_BTM)")
plt.title("Sensibilité au strike (call asiatique)")
plt.grid(True)
plt.show()


In [ ]:
def BS_option_price(t, St, K, T, r, sigma, opt_type):
    """
    Prix Black–Scholes d'une option européenne.

    opt_type : "call" ou "put"
    """
    tau = T - t
    if tau <= 0:
        return max(St - K, 0.0) if opt_type == "call" else max(K - St, 0.0)

    d1 = (np.log(St / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)

    if opt_type == "call":
        price = St * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * tau) * norm.cdf(-d2) - St * norm.cdf(-d1)
    return price


In [ ]:
range_price = np.array([57830 * (0.95 + 0.01 * j) for j in range(10)])

asian_call = []
euro_call = []

for K_val in range_price:
    asian_price = HW_BTM(
        strike_type="fixed",
        option_type="C",
        S0=57830,
        K=K_val,
        r=0.01,
        sigma=0.05,
        T=1.0,
        step=10,
        M=4
    )
    euro_price = BS_option_price(
        t=0.0,
        St=57830,
        K=K_val,
        T=1.0,
        r=0.01,
        sigma=0.05,
        opt_type="call"
    )
    asian_call.append(asian_price)
    euro_call.append(euro_price)

plt.figure(figsize=(10, 5))
plt.plot(range_price, asian_call, color="red", marker="o",
         label="Call asiatique arithmétique (fixed strike)")
plt.plot(range_price, euro_call, color="blue", marker="x",
         label="Call européen (Black–Scholes)")
plt.xlabel("Strike K")
plt.ylabel("Prix de l'option")
plt.title("Call asiatique vs call européen")
plt.legend()
plt.grid(True)
plt.show()

asian_put = []
euro_put = []

for K_val in range_price:
    asian_price = HW_BTM(
        strike_type="fixed",
        option_type="P",
        S0=57830,
        K=K_val,
        r=0.01,
        sigma=0.05,
        T=1.0,
        step=10,
        M=4
    )
    euro_price = BS_option_price(
        t=0.0,
        St=57830,
        K=K_val,
        T=1.0,
        r=0.01,
        sigma=0.05,
        opt_type="put"
    )
    asian_put.append(asian_price)
    euro_put.append(euro_price)

plt.figure(figsize=(10, 5))
plt.plot(range_price, asian_put, color="red", marker="o",
         label="Put asiatique arithmétique (fixed strike)")
plt.plot(range_price, euro_put, color="blue", marker="x",
         label="Put européen (Black–Scholes)")
plt.xlabel("Strike K")
plt.ylabel("Prix de l'option")
plt.title("Put asiatique vs put européen")
plt.legend()
plt.grid(True)
plt.show()
